# 4.1 The Y-86 ISA

### 4.1.1 Programmer-Visible State

The Y86-64 program can modify some part of the processor state. This is referred to as the *programmer visible state* where the programmer is someone writing programs in assembly.    
The state for Y86 is similar to that for x86. There are 15 program registers. There are three single bit condition codes for storing information, CF, ZF, OF.   
The **memory** is conceptually a large array of bytes. The Y84 programs reference memory locations using **virtual adresses**. Hardware and OS software translate these into actual or **physical addresses**. The status code indicates the state of program execution and indicates whether an exception has occurred.

### 4.1.2 Y86 Instructions

x86 movq instructions are split into irmovq, rrmovq, mrmovq, and rmmovq, the source isis either i, r, or m. The destination is r or m/ There are four integer operation instructions, operating only on reigster data. They set three condition codes, ZF, SF, and OF.
  
This is for Y86   

There are four integer operation instructions addq, subq, addq, and xorq.   

There are seven jump instructions, six conditional moves, and pushq, popq, and call. THere is even the halt instruction.

# 4.2 Logic Design and HCL

HCL is hardware control language, the language that is used to describe the control logic of the different processor designs.

### 4.2.1 Logic Gates

Logic gates are the basic computing elements for digital circuits and generate an ouput equal to a boolean function of the bit values of the inputs.   
Logic gates are always active.

### 4.2.2 Combinational Circuits

By assembling a number of logic gates, we can construct computational blocks known as combinational circuits.  
Outputs of two or more logic gates cannot be connected together, and the circuits must be acyclic for a certain circuit to be a combinational circuit.    

### 4.2.3 HCL expressions and Circuits

Typically, circuits are deisnged to operate on data words. These are groups of signals that represent integers or other control patterns.

### 4.2.5 Memory and Clocking

Circuits do not store information, they react to signals at their inputs. To create sequential circuits, we must introduce devices that sotre information.    
They will be controlled by a single **clock**, which determines when new values are loaded into the devices

**Clocked Registers** (or registers) store individual words or bits. The clock signal controls the loading and value of the register.      
  
**Random access memories** (or memories) store multiple words using an address to select which word should be read or written. Examples include the virtual memory system or the register file, in which the registers serve as addresses.

The register file has two read ports, and one write port. This allows multiple read and write operations to take place 